In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os

## Steps Before

1. Create a fork of the pa data repo
2. Create a copy of the counties folder for 2018 (referenced in the "path-in" below)
    - This allows me to import clean versions of the file

## Load Files

Note: When I load in files, I filter down to certain columns so I can work with the entire state at once. In order to be able to rejoin the data afterwards, I create an index on every load and keep that as well. When I export the files below, if its applicable to the county, I will rejoin any other columns back to the data I've modified

In [2]:
path_in = './raw-from-source/Election_Results/openelections-data-pa/2018/counties_copy/'
path_out = './raw-from-source/Election_Results/openelections-data-pa/2018/counties/'

In [3]:
combined_files = []

for file in os.listdir(path_in):
    if "__general" in file:
        test_load = pd.read_csv(path_in+file)
        test_load.reset_index(drop = False, inplace = True)
        test_load = test_load[["index","county","precinct","office","district","party","candidate","votes"]]
        combined_files.append(test_load)
        
pa_election = pd.concat(combined_files)
pa_election.reset_index(inplace = True, drop = True)

In [4]:
pa_election[pa_election["county"]=="Franklin"]

,index,county,precinct,office,district,party,candidate,votes
202108,0,Franklin,Antrim Twp - First Dist,Registered Voters,NaN,NaN,NaN,1723
202109,1,Franklin,Antrim Twp - First Dist,Ballots Cast,NaN,NaN,NaN,1092
202110,2,Franklin,Antrim Twp - First Dist,Straight Party,NaN,DEM,Democratic,96
202111,3,Franklin,Antrim Twp - First Dist,Straight Party,NaN,REP,Republican,611
202112,4,Franklin,Antrim Twp - First Dist,Straight Party,NaN,GRN,Green,0
...,...,...,...,...,...,...,...,...
203850,1742,Franklin,West End Shippensburg,U.S. House,13.0,DEM,Brent Ottaway,261
203851,1743,Franklin,West End Shippensburg,U.S. House,13.0,REP,John Joyce,221
203852,1744,Franklin,West End Shippensburg,U.S. House,13.0,NaN,Write - In,5
203853,1745,Franklin,West End Shippensburg,State House,89.0,REP,Rob Kauffman,296


## File Cleaning

In [5]:
# Clean up alternatives for the State House Office
pa_election.loc[pa_election["office"]=='REPRESENTATIVE IN THE GEN. ASSEMBLY 16TH DISTRICT',"district"] = 16.0
pa_election.loc[pa_election["office"]=='REPRESENTATIVE IN THE GEN. ASSEMBLY 15TH DISTRICT',"district"] = 15.0
pa_election.loc[pa_election["office"]=='REPRESENTATIVE IN THE GEN. ASSEMBLY 10TH DISTRICT',"district"] = 10.0
pa_election.loc[pa_election["office"]=='REPRESENTATIVE IN THE GEN. ASSEMBLY 14TH DISTRICT',"district"] = 14.0
pa_election.loc[pa_election["office"]=='SENATOR IN THE GENERAL ASSEMBLY 46TH DISTRICT',"district"] = 46.0

pa_election.loc[pa_election["office"]=='REPRESENTATIVE IN THE GEN. ASSEMBLY 16TH DISTRICT',"office"] = 'State House'
pa_election.loc[pa_election["office"]=='REPRESENTATIVE IN THE GEN. ASSEMBLY 15TH DISTRICT',"office"] = 'State House'
pa_election.loc[pa_election["office"]=='REPRESENTATIVE IN THE GEN. ASSEMBLY 10TH DISTRICT',"office"] = 'State House'
pa_election.loc[pa_election["office"]=='REPRESENTATIVE IN THE GEN. ASSEMBLY 14TH DISTRICT',"office"] = 'State House'
pa_election.loc[pa_election["office"]=='SENATOR IN THE GENERAL ASSEMBLY 46TH DISTRICT',"office"] = 'State Senate'


In [6]:
# Clean up a handful of other offices
pa_election.loc[pa_election["office"]=='State Senate�',"office"] = 'State Senate'
pa_election.loc[pa_election["office"]=='U.S. House†',"office"] = 'U.S. House'
pa_election.loc[pa_election['office']=='REPRESENTATIVE IN CONGRESS',"office"] = 'U.S. House'
pa_election.loc[pa_election['office']=='SENATOR IN THE GENERAL ASSEMBLY',"office"] = "State Senate"
pa_election.loc[pa_election['office']=='SENATOR IN THE GENERAL ASSEMBLY 147',"office"] = "State Senate"
pa_election.loc[pa_election['office']=='REPRESENTATIVE IN THE GENERAL ASSEMBLY','office'] = "State House"
pa_election.loc[pa_election['office']=='REPRESENTATIVE IN CONGRESS DISTRICT','office'] = 'U.S. House'

In [7]:
# In a handful of precincts, the State Senate district and candidates are recorded incorrectly
su34_list = ['0002 Barree Township',
 '0004 Brady Township',
 '0015 Henderson Township',
 '0018 Huntingdon/1st District',
 '0019 Huntingdon/2nd District',
 '0020 Huntingdon/3rd District',
 '0021 Huntingdon/4th District',
 '0022 Huntingdon/5th District',
 '0023 Huntingdon/6th District',
 '0024 Jackson Township',
 '0027 Logan Township',
 '0028 Mapleton Borough',
 '0031 Miller Township',
 '0036 Oneida Township',
 '0039 Petersburg Borough',
 '0048 Smithfield Township',
 '0057 West Township']

In [8]:
pa_election.loc[(pa_election["county"]=="Huntingdon")&
            (pa_election["office"]=="State Senate")&
            (pa_election["precinct"].isin(su34_list)),"district"] = 34.0

In [9]:
pa_election.loc[(pa_election["county"]=="Huntingdon")&
            (pa_election["office"]=="State Senate")&
            (pa_election["precinct"].isin(su34_list))&
            (pa_election["party"]=="REP"), "candidate"] = "JACOB D CORMAN III"

In [10]:
pa_election.loc[(pa_election["county"]=="Huntingdon")&
            (pa_election["office"]=="State Senate")&
            (pa_election["precinct"].isin(su34_list))&
            (pa_election["party"]=="DEM"), "candidate"] = "EZRA NANES"

In [11]:
# Where needed, fill out the party info
pa_election.loc[pa_election["candidate"]=="EMILY BEST (DEM)", "party"] = "DEM"
pa_election.loc[pa_election["candidate"]=="EMILY BEST (DEM)", "candidate"] = "EMILY BEST"

pa_election.loc[pa_election["candidate"]=="JUDY WARD (REP)", "party"] = "REP"
pa_election.loc[pa_election["candidate"]=="JUDY WARD (REP)", "candidate"] = "JUDY WARD"

In [12]:
# Final cleaning, confirmed this was state house
pa_election.loc[pa_election["office"]=="General Assembly","office"] = "State House"

## Clean up the candidate / strings

Note: The one "controversial" move here might be changing "DEM/REP":"REP". It seems like PA lists "DEM/REP" when there is only one of the two-party candidates represented. I set it to REP here and clean it up as needed.

In [13]:
pa_election.loc[:,'candidate'] = pa_election.loc[:,'candidate'].str.upper().str.strip()
pa_election.loc[:,'party'] = pa_election.loc[:,'party'].str.upper()

### Note: There are two candidates listed with "DEM/REP" as the party - for standardization purposes, I researched them on ballotpedia and selected their appropriate party. 

### If needed / desired, I can undo this step

In [14]:
pa_election.loc[pa_election["party"]=="DEM/REP"]["candidate"].unique()

array(['AARON BERNSTINE', 'TOMMY SANKEY'], dtype=object)

In [15]:
pa_election.loc[pa_election["candidate"]=='ELIZABETH BOOK', "party"] = "IND"
party_changes_dict = {"DEMOCRATIC":"DEM",
                      "REPUBLICAN":"REP",
                      "LIBERTARIAN":"LIB",
                      "GREEN":"GRN",
                      "GR":"GRN",
                      "GRE":"GRN",
                      "DEMOCRAT":"DEM",
                      "DEM/REP":"REP",
                      '`DEM':'DEM'}
pa_election["party"] = pa_election["party"].map(party_changes_dict).fillna(pa_election["party"])

In [16]:
# Fill in missing party information - Used Ballotpedia for this
# Also one candidate name cleaning
pa_election.loc[pa_election["candidate"]=='BOB CASEY JR.','party'] = "DEM"
pa_election.loc[pa_election["candidate"]=='BRENT OTTAWAY','party'] = "DEM"

pa_election.loc[pa_election["candidate"]=='DALE R. KERNS JR.','party'] = "LIB"
pa_election.loc[pa_election["candidate"]=='DAN MOUL','party'] = "REP"
pa_election.loc[pa_election["candidate"]=="DANIEL WILSON", "candidate"] = "DANIEL WILSON (W)"
pa_election.loc[pa_election["candidate"]=="JOHN JOYCE","party"] = "REP"
pa_election.loc[pa_election["candidate"]=="JOHN M. WALDENBERGER","party"] = "LIB"
pa_election.loc[pa_election["candidate"]=="KEN V. KRAWCHUK / K. S. SMITH","party"] = "LIB"
pa_election.loc[pa_election["candidate"]=="LOU BARLETTA", "party"] = "REP"
pa_election.loc[pa_election["candidate"]=="MARC BOZZACCO","party"] = "LIB"
pa_election.loc[pa_election["candidate"]=="TOM WOLF", "party"] = "DEM"
pa_election.loc[pa_election["candidate"]=="TOM WOLF / JOHN FETTERMAN", "party"] = "DEM"
pa_election.loc[pa_election["candidate"]=="MARTY QUALLY", "party"] = "DEM"
pa_election.loc[pa_election["candidate"]=="TOM WOLF", "party"] = "DEM"
pa_election.loc[pa_election["candidate"]=="MATTHEW NELSON", "party"] = "DEM"
pa_election.loc[pa_election["candidate"]=="NEALE GALE", "party"] = "GRN"
pa_election.loc[pa_election["candidate"]=="PAUL GLOVER / J. BOWSER BOSTICK", "party"] = "GRN"
pa_election.loc[pa_election["candidate"]=="SCOTT R. WAGNER / JEFF BARTOS", "party"] = "REP"
pa_election.loc[pa_election["candidate"]=="TORREN ECKER", "party"] = "REP"

In [17]:
# Add in missing State House info
pa_election.loc[(pa_election["district"].isna())&(pa_election["office"]=="State House")&(pa_election["precinct"]=="Upper Dublin 4-2"),"district"]=153

## Various Typo Cleaning

In [18]:
pa_election.loc[(pa_election["precinct"]=="Limerick 3") &
                        (pa_election["office"]=="State House") &
                         (pa_election["party"]=="DEM"),"candidate"] = "JOE CIRESI"

pa_election.loc[(pa_election["precinct"]=="Limerick 3") &
                        (pa_election["office"]=="State House") &
                         (pa_election["party"]=="REP"),"candidate"] = "THOMAS J QUIGLEY"

pa_election.loc[(pa_election["precinct"].isin(['Upper Merion Bel 2', 'Upper Merion Bel 3'])) &
                        (pa_election["office"]=="State House"), "district"] = 149.0

pa_election.loc[(pa_election["precinct"].isin(['Lower Merion 4-2'])) &
                        (pa_election["office"]=="State House"),"district"] = 166.0

In [19]:
pa_election.loc[(pa_election["precinct"].isin(['Pottstown 2-2'])) &
                        (pa_election["office"]=="State House") &
                         (pa_election["party"]=="REP"),"candidate"] = 'TIM HENNESSEY'

pa_election.loc[(pa_election["precinct"].isin(['Pottstown 2-2'])) &
                        (pa_election["office"]=="State House") &
                         (pa_election["party"]=="DEM"),"candidate"] = 'PAM HACKER'

In [20]:
pa_election.loc[(pa_election["precinct"].isin(['Lower Gwynedd 2-1'])) &
                        (pa_election["office"]=="State House"),"district"] = 151.0

In [21]:
pa_election.loc[(pa_election["precinct"].isin(['East Norriton 1-4', 'East Norriton 2-1'])) &
                        (pa_election["office"]=="State House") &
                         (pa_election["party"]=="DEM"),"candidate"] = 'MATT BRADFORD'

pa_election.loc[(pa_election["precinct"].isin(['East Norriton 1-4', 'East Norriton 2-1'])) &
                        (pa_election["office"]=="State House") &
                         (pa_election["party"]=="REP"),"candidate"] = 'CHRIS MUNDIATH'

In [22]:
pa_election.loc[(pa_election["precinct"].isin(['Pottstown 2-2'])) &
                        (pa_election["office"]=="State Senate") &
                         (pa_election["party"]=="DEM"),"candidate"] = 'LINDA FIELDS'

pa_election.loc[(pa_election["precinct"].isin(['Pottstown 2-2'])) &
                        (pa_election["office"]=="State Senate") &
                         (pa_election["party"]=="REP"),"candidate"] = 'BOB MENSCH'

In [23]:
pa_election.loc[(pa_election["precinct"].isin(['Limerick 3'])) &
                        (pa_election["office"]=="State Senate") &
                         (pa_election["party"]=="DEM"),"candidate"] = 'KATIE MUTH'

pa_election.loc[(pa_election["precinct"].isin(['Limerick 3'])) &
                        (pa_election["office"]=="State Senate") &
                         (pa_election["party"]=="REP"),"candidate"] = 'JOHN RAFFERTY'

In [24]:
pa_election.loc[(pa_election["precinct"].isin(['Horsham 2-3'])) &
                        (pa_election["candidate"].isin(["BRIAN FITZPATRICK", 'SCOTT WALLACE'])),"district"] = 1.0



In [25]:
# Need to grab the other votes beofre the below
special_15_precs = list(pa_election.loc[(pa_election["candidate"].isin(["SUSAN WILD"])) & 
                        (pa_election["district"]==15.0)]['precinct'].unique())

pa_election.loc[(pa_election["precinct"].isin(special_15_precs) &
                         (pa_election["district"]==15.0)),"office"] = "Special U.S. House"

In [26]:
# Need to grab the other votes beofre the below
special_15_precs = list(pa_election.loc[(pa_election["candidate"].isin(["MARY GAY SCANLON"])) & 
                        (pa_election["district"]==7.0)]['precinct'].unique())

pa_election.loc[(pa_election["precinct"].isin(special_15_precs) &
                         (pa_election["district"]==7.0)),"office"] = "Special U.S. House"

In [27]:
pa_election.loc[pa_election["candidate"]=="Other", "party"] = "OTH"
pa_election.loc[pa_election["candidate"]=="Other", "candidate"] = "WRITE IN"

In [28]:
pa_election.loc[(pa_election["office"]=="State House") &
                        (pa_election["district"]==108.0) &
                        (pa_election["party"]=="REP") &
                        (pa_election['candidate']=="WRITE IN VOTES"), 'party'] = "OTH"

In [29]:
pa_election.loc[(pa_election["candidate"]=="BRAD ROE"), "candidate"] = "BRAD ROAE"

In [30]:
pa_election.loc[(pa_election["candidate"]=="DOYLE HALLEY"),"candidate"] = "DOYLE HEFFLEY"

In [31]:
pa_election.loc[(pa_election["candidate"]=="AARON BERNSTINE"),"party"] = "REP"

In [32]:
pa_election.loc[(pa_election["precinct"]=='Cornplanter Twp') & 
                    (pa_election["candidate"]=="R LEE JAMES"),"votes"] = 588

In [33]:
pa_election.loc[(pa_election["office"]=='State Senate') &
                (pa_election["precinct"]=="10 East Side Borough") &
                (pa_election["party"]=="DEM"), "votes"] = 40.0

In [34]:
pa_election.loc[(pa_election["precinct"]=="Franklin Twp") &
               (pa_election["office"]=="Governor") &
               (pa_election["party"]=="REP"), "votes"] = 273

In [35]:
pa_election.loc[(pa_election["precinct"]=="Waynesboro Boro 3-2") &
               (pa_election["office"]=="Governor") &
               (pa_election["party"]=="REP"), "votes"] = 282

In [36]:
pa_election.loc[(pa_election["precinct"]=="BelleVernon") & 
               (pa_election["office"]=="State Senate") &
               (pa_election["votes"]==160), 'candidate'] = 'PAT STEFANO'

pa_election.loc[(pa_election["precinct"]=="BelleVernon") & 
               (pa_election["office"]=="State Senate") &
               (pa_election["votes"]==160),"party"] = "REP"

In [37]:
pa_election.loc[(pa_election["precinct"]=="Blooming Valley Boro") & 
               (pa_election["office"]=="State Senate") &
               (pa_election["party"]=="DEM"), "votes"] = 33.0

pa_election.loc[(pa_election["precinct"]=="Blooming Valley Boro") & 
               (pa_election["office"]=="State Senate") &
               (pa_election["party"]=="REP"), "votes"] = 130.0

In [38]:
pa_election.loc[(pa_election["precinct"]=="Rome Twp") & 
               (pa_election["office"]=="State House"), "district"] = 65.0

In [39]:
pa_election.loc[(pa_election["precinct"]=="Rome Twp") & 
               (pa_election["office"]=="State House") &
               (pa_election["party"]=="REP"), "candidate"] = "KATHY L RAPP"

pa_election.loc[(pa_election["precinct"]=="Rome Twp") & 
               (pa_election["office"]=="State House") &
               (pa_election["party"]=="REP"), "votes"] = 344

pa_election.loc[(pa_election["precinct"]=="Rome Twp") & 
               (pa_election["office"]=="State House") &
               (pa_election["candidate"]=="WRITE-INS"), "votes"] = 0

In [40]:
pa_election.loc[(pa_election["county"]=="Crawford") & 
               (pa_election["candidate"]=="PARKE WENTLING") &
               (pa_election["precinct"]=="Summerhill Twp"), "votes"] = 332.0

pa_election.loc[(pa_election["county"]=="Crawford") & 
               (pa_election["candidate"]=="PARKE WENTLING") &
               (pa_election["precinct"]=="W. Shenango Twp"), "votes"] = 133.0

In [41]:
pa_election.loc[(pa_election["county"]=="Crawford") & 
               (pa_election["candidate"]=="PARKE WENTLING") &
               (pa_election["precinct"]=="Steuben Twp"), "district"] = 65.0

pa_election.loc[(pa_election["county"]=="Crawford") & 
               (pa_election["candidate"]=="PARKE WENTLING") &
               (pa_election["precinct"]=="Steuben Twp"), "candidate"] = "KATHY L RAPP"


## Candidate Name Standardization

Note: Many of the changes below rely upon this standardization, the only controversial change might be, though I don't think much is lost dropping the plural
'WRITE IN VOTES':'WRITE-IN', 'Write-ins':'WRITE-IN',

In [42]:
cleaning_names_dict = {'BOB CASEY JR': 'BOB CASEY, JR',
 'BOB CASEY, JR.': 'BOB CASEY, JR',
 'BOB CASEY JR.': 'BOB CASEY, JR',
 'BOB CASEY': 'BOB CASEY, JR',
 'CASEY, JR., BOB': 'BOB CASEY, JR',
 'ROBERT CASEY JR.': 'BOB CASEY, JR',
 'LOU BARLETTA JR': 'LOU BARLETTA',
 'BARLETTA, LOU': 'LOU BARLETTA',
 'GALE, NEAL': 'NEAL GALE',
 'NEALE GALE': 'NEAL GALE',
 'DALE R KERNS JR': 'DALE R KERNS, JR',
 'DALE R. KERNS, JR.': 'DALE R KERNS, JR',
 'DALE R. KERNS JR': 'DALE R KERNS, JR',
 'DALE R. KERNS JR.': 'DALE R KERNS, JR',
 'DALE KERNS JR': 'DALE R KERNS, JR',
 'DALE R. KERNS, JR': 'DALE R KERNS, JR',
 'KERNS, JR., DALE R.': 'DALE R KERNS, JR',
 'DALE R KEARNS, JR': 'DALE R KERNS, JR',
 'DALE KERNS': 'DALE R KERNS, JR',
 'WOLF/FETTERMAN': 'TOM WOLF',
 'WOLF / FETTERMAN': 'TOM WOLF',
 'TOM WOLF/JOHN FETTERMAN': 'TOM WOLF',
 'TOM WOLF AND JOHN FETTERMAN': 'TOM WOLF',
 'WOLF, TOM': 'TOM WOLF',
 'TOM WOLF / JOHN FETTERMAN': 'TOM WOLF',
 'WOLF\\FETTERMAN': 'TOM WOLF',
 'TOM WOLF JOHN FETTERMAN': 'TOM WOLF',
 'TOM WOLF, GOVERNOR': 'TOM WOLF',
 'WAGNER/BARTOS': 'SCOTT R WAGNER',
 'SCOTT R. WAGNER': 'SCOTT R WAGNER',
 'WAGNER / BARTOS': 'SCOTT R WAGNER',
 'SCOTT R. WAGNER/JEFF BARTOS': 'SCOTT R WAGNER',
 'WAGNER/ BARTOS': 'SCOTT R WAGNER',
 'SCOTT R WAGNER AND JEFF BARTOS': 'SCOTT R WAGNER',
 'SCOTT WAGNER': 'SCOTT R WAGNER',
 'WAGNER, SCOTT R.': 'SCOTT R WAGNER',
 'SCOTT R. WAGNER / JEFF BARTOS': 'SCOTT R WAGNER',
 'WAGNER\\BARTOS': 'SCOTT R WAGNER',
 'SCOTT R. WAGNER JEFF BARTOS': 'SCOTT R WAGNER',
 'SCOTT R WAGNER, GOVERNOR': 'SCOTT R WAGNER',
 'GLOVER/BOWSER-BOSTIC': 'PAUL GLOVER',
 'GLOVER / BOWSER-BOSTICK': 'PAUL GLOVER',
 'GLOVER/BOWSER-BOSTICK': 'PAUL GLOVER',
 'GLOVER/BOWSER-BOS': 'PAUL GLOVER',
 'PAUL GLOVER/JOCOLYN BOWER-BOSTICK': 'PAUL GLOVER',
 'GLOVER / BOWSER-BOS': 'PAUL GLOVER',
 'GLOVER, PAUL': 'PAUL GLOVER',
 'PAUL GLOVER / J. BOWSER BOSTICK': 'PAUL GLOVER',
 'GLOVER\\BOWSERBOSTICK': 'PAUL GLOVER',
 'GLOVER/BOWSERBOS': 'PAUL GLOVER',
 'PAUL GLOVER JOCOLYN BOWSER-BOSTICK': 'PAUL GLOVER',
 'GLOVER/BOSTICK': 'PAUL GLOVER',
 'PAUL GLOVER/J. BOWSER-BOSTICK': 'PAUL GLOVER',
 'GLOVER / BOWSER BOSTICK': 'PAUL GLOVER',
 'PAUL GLOVER, GOVERNOR': 'PAUL GLOVER',
 'GLOVER / BOSTICK': 'PAUL GLOVER',
 'KRAWCHUK/SMITH': 'KEN V KRAWCHUK',
 'KEN V. KRAWCHUK': 'KEN V KRAWCHUK',
 'KRAWCHUK / SMITH': 'KEN V KRAWCHUK',
 'KEN V. KRAWCHUK/K. S. SMITH': 'KEN V KRAWCHUK',
 'KEN KRAWCHUK': 'KEN V KRAWCHUK',
 'KRAWCHUK/ SMITH': 'KEN V KRAWCHUK',
 'KRAWCHUK, KEN V.': 'KEN V KRAWCHUK',
 'KEN V. KRAWCHUK / K. S. SMITH': 'KEN V KRAWCHUK',
 'KRAWCHUK\\SMITH': 'KEN V KRAWCHUK',
 'KEN V. KRAWCHUK KATHLEEN S. SMITH': 'KEN V KRAWCHUK',
 'KRAWCHUK /SMITH': 'KEN V KRAWCHUK',
 'KEN V. KRAWCHUK/K.S. SMITH': 'KEN V KRAWCHUK',
 'KEN V KRAWCHUK, GOVERNOR': 'KEN V KRAWCHUK',
 'RON DI NICOLA': 'RON DINICOLA',
 'EBERT G. BILL BEEMAN': 'EBERT G BILL BEEMAN',
 'EBERT G. "BILL" BEEMAN': 'EBERT G BILL BEEMAN',
 'GLENN THOMPSON': 'GLENN GT THOMPSON',
 'R LEE JAMES': 'R. LEE JAMES',
 'GUY L. RESCHENTHALER': 'GUY RESCHENTHALER',
 'JOSHUA R. NULPH': 'JOSHUA R NULPH',
 'JOSEPH A. PETRARCA': 'JOSEPH A PETRARCA',
 'BRENT M. OTTAWAY': 'BRENT OTTAWAY',
 'MICHAEL P. REESE': 'MIKE REESE',
 'PAMELA GERARD': 'PAM GERARD',
 'PATRICK J. STEFANO': 'PAT STEFANO',
 'JENNIFER R. RAGER-KAY': 'JENNIFER RAGER-KAY',
 'LYNDA J. SCHLEGEL-CULVER': 'LYNDA J. SCHLEGEL-CULVERLVER',
 'LYNDA J. SCHLEGEL CULVER': 'LYNDA J. SCHLEGEL-CULVERLVER',
 'LLOYD K. SMUCKER': 'LLOYD K SMUCKER',
 'THOMAS P MURT': 'THOMAS P. MURT',
 'KEVIN J BOYLE': 'KEVIN J. BOYLE',
 'MARIA P. DONATUCCI': 'MARIA P DONATUCCI',
 'SHARON M. GUIDI': 'SHARON GUIDI',
 'JASON A. ORTITAY': 'JASON ORTITAY',
 'CARL METZGAR': 'CARL WALKER METZGAR',
 'MARTIN T CAUSER': 'MARTIN T. CAUSER',
 'JOHN T. YUDICHAK': 'JOHN T YUDICHAK',
 'JOHN J. SWEENEY': 'JOHN J SWEENEY',
 'JOHN BLAKE': 'JOHN P BLAKE',
 'JOHN P. BLAKE': 'JOHN P BLAKE',
 'SANDRA T. SALAS': 'SANDRA TERESA SALAS',
 'KATHY L. RAPP': 'KATHY L RAPP',
 'TIM MAHONEY': 'TIMOTHY S. MAHONEY',
 'BRYAN BARBIN': 'BRYAN E. BARBIN',
 'JIM RIGBY': 'JAMES PATRICK RIGBY',
 'MICHAEL PEIFER': 'MIKE PEIFER',
 'JUSTIN J. SIMMONS': 'JUSTIN J SIMMONS',
 'RYAN E. MACKENZIE': 'RYAN E MACKENZIE',
 'MICHAEL BLICHAR': 'MICHAEL BLICHAR JR',
 'CAMERA C. BARTOLOTTA': 'CAMERA BARTOLOTTA',
 'STEVEN M. TOPRANI': 'STEVEN M TOPRANI',
 'PAMELA SNYDER': 'PAM SNYDER',
 'ELIZABETH L. ROHANNA MCCLURE': 'BETSY ROHANNA MCCLURE',
 'FIELDS, LINDA':'LINDA FIELDS',
  'MENSCH, BOB':'BOB MENSCH',
 'COLLETT, MARIA':'MARIA COLLETT',
  'GREENLEAF, JR., STEWART':'STEWART GREENLEAF JR',
 'THOMAS J MCGARRIGLE SR': 'THOMAS J. MCGARRIGLE SR',
 'MARIO M. SCAVELLO': 'MARIO SCAVELLO',
 'CLINT OWLET': 'CLINT OWLETT',
 'JOHN M WALDENBERGER': 'JOHN M. WALDENBERGER',
 'FITZPATRICK, BRIAN':'BRIAN FITZPATRICK',
 'WALLACE, SCOTT':'SCOTT WALLACE',
 'WRITE IN VOTES':'WRITE-IN',
 'WRITE IN':'WRITE-IN',
 'Write-ins':'WRITE-IN',
 "WRITE - IN":'WRITE-IN',
'GALLOWAY, JOHN T.': "JOHN T. GALLOWAY",
'SPOSATO, ANTHONY J.': 'ANTHONY J. SPOSATO',
 'GALLAGHER, RYAN W.':'RYAN W. GALLAGHER',
 'SANTARSIERO, STEVE': 'STEVE SANTARSIERO',
 'QUINN, MARGUERITE': 'MARGUERITE QUINN',
 'ULLMAN, WENDY': 'WENDY ULLMAN',
 'FLOOD, JOE': 'JOE FLOOD',
 'DAVIS, TINA': 'TINA DAVIS',
 'TOMLINSON, ROBERT TOMMY': 'ROBERT TOMMY TOMLINSON',
 'LAMB, JIMMY': 'JIMMY LAMB',
 'DIGIROLAMO, GENE': 'GENE DIGIROLAMO',
 'DIXON, ANDREW': 'ANDREW DIXON',
 'SCHROEDER, MEGHAN': 'MEGHAN SCHROEDER',
 'BUCK, MEREDITH': 'MEREDITH BUCK',
 'POLINCHOCK, TODD': 'TODD POLINCHOCK',
 'KLINE, BRIAN': 'BRIAN KLINE',
 'STAATS, CRAIG': 'CRAIG STAATS',
 'LAREAU, LAUREN': 'LAUREN LAREAU',
 'FARRY, FRANK': 'FRANK FARRY',
 'WARREN, PERRY': 'PERRY WARREN',
 'TAI, HELEN': 'HELEN TAI',
 'THOMAS, WENDI': 'WENDI THOMAS',
 'DOMINIC J. PIROCCHI':'DOMINIC J PIROCCHI',
 'DARCELLE L SLAPPY':'DARCELLE L. SLAPPY'}

In [43]:
pa_election['candidate'] = pa_election['candidate'].map(cleaning_names_dict).fillna(pa_election['candidate'])

In [44]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Franconia 8") & 
(pa_election["candidate"]=="MADELEINE DEAN"),"district"] = 1.0

pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Franconia 8") & 
(pa_election["candidate"]=="MADELEINE DEAN"),"candidate"] = "SCOTT WALLACE"

pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Franconia 8") & 
(pa_election["candidate"]=="DAN DAVID"),"district"] = 1.0

pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Franconia 8") & 
(pa_election["candidate"]=="DAN DAVID"),"candidate"] = "BRIAN FITZPATRICK"

In [45]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Lower Gwynedd 1-3") &
(pa_election["candidate"]=="DAN DAVID"), "votes"] = 454.0

In [46]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Upper Providence Mingo 1") &
(pa_election["candidate"]=="MADELEINE DEAN"), "votes"] = 814.0

In [47]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Pottstown 1-2") &
(pa_election["candidate"]=="MADELEINE DEAN"), "votes"] = 231.0

In [48]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Lower Merion 13-1") &
(pa_election["candidate"]=="MADELEINE DEAN"), "votes"] = 635.0

In [49]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Lower Merion 9-1") &
(pa_election["candidate"]=="MADELEINE DEAN"), "votes"] = 579.0

In [50]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="New Hanover 3") &
(pa_election["candidate"]=="SCOTT R WAGNER"),"votes"] = 794.0

In [51]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Montgomery 7") &
(pa_election["candidate"]=="TOM WOLF"), "votes"] = 1477.0

In [52]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Abington 13-3") &
(pa_election["candidate"]=="LOU BARLETTA"), "votes"] = 160.0

In [53]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Cheltenham 6-3") &
(pa_election["candidate"]=="SCOTT R WAGNER"), "votes"] = 125.0

In [54]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Cheltenham 6-3") &
(pa_election["candidate"]=="DAN DAVID"), "votes"] = 138.0

In [55]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Cheltenham 6-3")&
(pa_election["candidate"]=="WRITE-INS")&
(pa_election["office"]=="U.S. House"), "votes"] = 2.0

In [56]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Jenkintown 1") &
(pa_election["candidate"]=="SCOTT R WAGNER"), "votes"] = 139.0

In [57]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Franconia 2-4th") &
(pa_election["candidate"]=="DALE R KERNS, JR"), "votes"] = 12.0

In [58]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Franconia 2-4th") &
(pa_election["candidate"]=="NEAL GALE"), "votes"] = 8.0

In [59]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Abington 4-2") &
(pa_election["candidate"]=="DALE R KERNS, JR"), "votes"] = 13.0

In [60]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Abington 4-2") &
(pa_election["candidate"]=="NEAL GALE"), "votes"] = 7.0

In [61]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Abington 4-2") &
(pa_election["candidate"]=="LOU BARLETTA"), "votes"] = 178.0

In [62]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Lower Merion 5-2") &
(pa_election["candidate"]=="LOU BARLETTA"), "votes"] = 139.0

In [63]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Limerick 4") &
(pa_election["office"]=="U.S. House") &
(pa_election["candidate"]=="WRITE-INS"),"votes"] = 1.0

In [64]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Lower Providence 2-1") &
(pa_election["office"]=="U.S. House") &
(pa_election["candidate"]=="WRITE-INS"),"votes"] = 0.0

In [65]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Montgomery 2") &
(pa_election["office"]=="U.S. House") &
(pa_election["candidate"]=="WRITE-INS"),"votes"] = 2.0

In [66]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Plymouth 2-3 - 7th") &
(pa_election["office"]=="U.S. House") &
(pa_election["candidate"]=="WRITE-INS"),"votes"] = 1.0

In [67]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Upper Moreland 6-2") &
(pa_election["office"]=="U.S. House") &
(pa_election["candidate"]=="WRITE-INS"),"votes"] = 1.0

In [68]:
pa_election.loc[(pa_election["county"]=="Montgomery") &
(pa_election["precinct"]=="Abington 12-1") &
(pa_election["office"]=="U.S. House") &
(pa_election["candidate"]=="WRITE-INS"),"votes"] = 1.0

In [69]:
pa_election.loc[(pa_election["precinct"]=="Portage Boro Second Wd") &
               (pa_election["candidate"]=="TOM WOLF"),"votes"] = 135.0

In [70]:
pa_election.loc[(pa_election["precinct"]=="Portage Boro Second Wd") &
               (pa_election["candidate"]=="SCOTT R WAGNER"),"votes"] = 143.0

In [71]:
pa_election.loc[(pa_election["candidate"].isin(["MADELEINE DEAN",'DAN DAVID'])),"district"] = 4.0

In [72]:
pa_election.loc[(pa_election["candidate"]=='JOSEPH A PETRARCA'),"party"] = "DEM"

In [73]:
pa_election.loc[(pa_election["candidate"]=='JOHN M. WALDENBERGER'),"office"] = "State House"
pa_election.loc[(pa_election["candidate"]=='JOHN M. WALDENBERGER'),"district"] = 53.0


In [74]:
pa_election.loc[(pa_election["office"]=='State House') &
                (pa_election['district']==108.0) &
                (pa_election["candidate"]=="WRITE-IN"),"party"] = "OTH"

In [75]:
pa_election[pa_election["county"]=="Franklin"]

,index,county,precinct,office,district,party,candidate,votes
202108,0,Franklin,Antrim Twp - First Dist,Registered Voters,NaN,NaN,NaN,1723
202109,1,Franklin,Antrim Twp - First Dist,Ballots Cast,NaN,NaN,NaN,1092
202110,2,Franklin,Antrim Twp - First Dist,Straight Party,NaN,DEM,DEMOCRATIC,96
202111,3,Franklin,Antrim Twp - First Dist,Straight Party,NaN,REP,REPUBLICAN,611
202112,4,Franklin,Antrim Twp - First Dist,Straight Party,NaN,GRN,GREEN,0
...,...,...,...,...,...,...,...,...
203850,1742,Franklin,West End Shippensburg,U.S. House,13.0,DEM,BRENT OTTAWAY,261
203851,1743,Franklin,West End Shippensburg,U.S. House,13.0,REP,JOHN JOYCE,221
203852,1744,Franklin,West End Shippensburg,U.S. House,13.0,NaN,WRITE-IN,5
203853,1745,Franklin,West End Shippensburg,State House,89.0,REP,ROB KAUFFMAN,296


### Berks County U.S. House 7 Special Election

Need to add votes in for Cumru Township 2 and 3, these did not get added at all

In [76]:
pa_election[(pa_election["county"]=="Berks")&(pa_election["precinct"]=="Cumru Twp. 02")]

,index,county,precinct,office,district,party,candidate,votes
186820,30,Berks,Cumru Twp. 02,U.S. Senate,NaN,DEM,"BOB CASEY, JR",449
187024,234,Berks,Cumru Twp. 02,U.S. Senate,NaN,REP,LOU BARLETTA,575
187228,438,Berks,Cumru Twp. 02,U.S. Senate,NaN,GRN,NEAL GALE,6
187432,642,Berks,Cumru Twp. 02,U.S. Senate,NaN,LIB,"DALE R KERNS, JR",9
187636,846,Berks,Cumru Twp. 02,U.S. Senate,NaN,NaN,WRITE-INS,0
187840,1050,Berks,Cumru Twp. 02,Governor,NaN,DEM,TOM WOLF,466
188044,1254,Berks,Cumru Twp. 02,Governor,NaN,REP,SCOTT R WAGNER,558
188248,1458,Berks,Cumru Twp. 02,Governor,NaN,GRN,PAUL GLOVER,6
188452,1662,Berks,Cumru Twp. 02,Governor,NaN,LIB,KEN V KRAWCHUK,10
188656,1866,Berks,Cumru Twp. 02,Governor,NaN,NaN,WRITE-INS,0


In [77]:
pa_election.shape[0]

203855

In [78]:
pa_election.loc[pa_election.shape[0]] = [3503, "Berks", "Cumru Twp. 02", "Special U.S. House",7.0,"DEM","MARY GAY SCANLON",412]
pa_election.loc[pa_election.shape[0]] = [3504, "Berks", "Cumru Twp. 02", "Special U.S. House",7.0,"REP","PEARL KIM",581]
pa_election.loc[pa_election.shape[0]] = [3505, "Berks", "Cumru Twp. 02", "Special U.S. House",7.0,"LIB","SANDRA TERESA SALAS",14]
pa_election.loc[pa_election.shape[0]] = [3506, "Berks", "Cumru Twp. 02", "Special U.S. House",7.0,"GRN","BRIANNA JOHNSTON",11]

pa_election.loc[pa_election.shape[0]] = [3507, "Berks", "Cumru Twp. 03", "Special U.S. House",7.0,"DEM","MARY GAY SCANLON",344]
pa_election.loc[pa_election.shape[0]] = [3508, "Berks", "Cumru Twp. 03", "Special U.S. House",7.0,"REP","PEARL KIM",480]
pa_election.loc[pa_election.shape[0]] = [3509, "Berks", "Cumru Twp. 03", "Special U.S. House",7.0,"LIB","SANDRA TERESA SALAS",10]
pa_election.loc[pa_election.shape[0]] = [3510, "Berks", "Cumru Twp. 03", "Special U.S. House",7.0,"GRN","BRIANNA JOHNSTON",20]

pa_election.loc[pa_election.shape[0]] = [3511, "Berks", "Cumru Twp. 02", "Special U.S. House",7.0,"OTH","WRITE-INS",0]
pa_election.loc[pa_election.shape[0]] = [3512, "Berks", "Cumru Twp. 03", "Special U.S. House",7.0,"OTH","WRITE-INS",0]

In [79]:
pa_election[pa_election["county"]=="Franklin"]

,index,county,precinct,office,district,party,candidate,votes
202108,0,Franklin,Antrim Twp - First Dist,Registered Voters,NaN,NaN,NaN,1723
202109,1,Franklin,Antrim Twp - First Dist,Ballots Cast,NaN,NaN,NaN,1092
202110,2,Franklin,Antrim Twp - First Dist,Straight Party,NaN,DEM,DEMOCRATIC,96
202111,3,Franklin,Antrim Twp - First Dist,Straight Party,NaN,REP,REPUBLICAN,611
202112,4,Franklin,Antrim Twp - First Dist,Straight Party,NaN,GRN,GREEN,0
...,...,...,...,...,...,...,...,...
203850,1742,Franklin,West End Shippensburg,U.S. House,13.0,DEM,BRENT OTTAWAY,261
203851,1743,Franklin,West End Shippensburg,U.S. House,13.0,REP,JOHN JOYCE,221
203852,1744,Franklin,West End Shippensburg,U.S. House,13.0,NaN,WRITE-IN,5
203853,1745,Franklin,West End Shippensburg,State House,89.0,REP,ROB KAUFFMAN,296


### Montgomery County U.S. House 7 Special Election Cleaning

In [80]:
pa_election.loc[(pa_election["county"]=='Montgomery') &
           (pa_election["office"]=='SPECIAL ELECTION REPRESENTATIVE IN CONGRESS'),"office"] = "Special U.S. House"

In [81]:
pa_election.loc[(pa_election["county"]=='Montgomery') &
                (pa_election["office"]=="Special U.S. House"),"district"] = 7.0

#### For the Special U.S. House race, too many values were assigned to Plymouth 2-1, where they should be given to Plymouth 2-2

Note: I made the changes by votes here, so I didn't have to refer to particular indices. There were no write-in votes in either precinct, but I adjusted the write-in votes as well, but I had to refer to the index


In [82]:
pa_election.loc[(pa_election["county"]=="Montgomery")&
                (pa_election["office"]=="Special U.S. House")&
                (pa_election["precinct"]=="Plymouth 2-1")&
                (pa_election["votes"].isin([373,276,2])),"precinct"] = "Plymouth 2-2"

In [83]:
pa_election.loc[(pa_election["county"]=="Montgomery")&(pa_election['index']==7919),"precinct"] = "Plymouth 2-2"

### Fill in NaN candidates in the Special U.S. House Election in Montgomery Couty

In [84]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="Special U.S. House") &
            (pa_election["party"]=="DEM"),"candidate"] = "MARY GAY SCANLON"

In [85]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="Special U.S. House") &
            (pa_election["party"]=="REP"),"candidate"] = "PEARL KIM"

In [86]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="Special U.S. House") &
            (pa_election["party"]=="GRN"),"candidate"] = "BRIANNA JOHNSTON"

In [87]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="Special U.S. House") &
            (pa_election["party"]=="LIB"),"candidate"] = "SANDRA TERESA SALAS"

In [88]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="Special U.S. House") &
            (pa_election["party"].isna()),"candidate"] = "WRITE-INS"

### Montgomery County State House and State Senate Cleaning

In [89]:
# These votes need to be switched
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State House")&
            (pa_election["district"]==53)&
            (pa_election["precinct"]=="Lansdale 2-1")&
            (pa_election["candidate"]=="JOHN M. WALDENBERGER")&
            (pa_election["votes"]==193.0), "candidate"] = "BOB MENSCH"

pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State House")&
            (pa_election["district"]==53)&
            (pa_election["precinct"]=="Lansdale 2-1")&
            (pa_election["candidate"]=="BOB MENSCH")&
            (pa_election["votes"]==193.0), "office"] = "State Senate"

pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State Senate")&
            (pa_election["district"]==53)&
            (pa_election["precinct"]=="Lansdale 2-1")&
            (pa_election["candidate"]=="BOB MENSCH")&
            (pa_election["votes"]==193.0), "district"] = 24

pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State Senate")&
            (pa_election["district"]==24)&
            (pa_election["precinct"]=="Lansdale 2-1")&
            (pa_election["candidate"]=="BOB MENSCH")&
            (pa_election["votes"]==193.0), "party"] = "REP"

In [90]:
# Visual confirmation the switch worked
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State Senate")&
            (pa_election["precinct"]=="Lansdale 2-1")]

,index,county,precinct,office,district,party,candidate,votes
175234,829,Montgomery,Lansdale 2-1,State Senate,24,DEM,LINDA FIELDS,426.0
175235,830,Montgomery,Lansdale 2-1,State Senate,24,REP,BOB MENSCH,193.0
175236,831,Montgomery,Lansdale 2-1,State Senate,24,NaN,WRITE-INS,0.0


In [91]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State House")&
            (pa_election["candidate"]=='ANDY SZEKELY')&
            (pa_election["precinct"]=="Souderton 3"),"votes"] = 365.0

In [92]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State House")&
            (pa_election["candidate"]=='JOE WEBSTER')&
            (pa_election["precinct"]=="Lower Providence 1-2"), "votes"] = 753.0

In [93]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State House")&
            (pa_election["candidate"]=='NICK FOUNTAIN')&
            (pa_election["precinct"]=="Lower Providence 1-2"), "votes"] = 593.0

In [94]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State House")&
            (pa_election["precinct"]=="Lower Moreland 4-1")&
            (pa_election["party"]=="DEM"),"candidate"] = 'DARYL BOLING'

pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State House")&
            (pa_election["precinct"]=="Lower Moreland 4-1")&
            (pa_election["party"]=="REP"),"candidate"] = 'THOMAS P. MURT'

In [95]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State Senate")&
            (pa_election["precinct"]=="Lower Moreland 4-1")&
            (pa_election["party"]=="REP"), "candidate"] = "STEWART GREENLEAF JR"

pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State Senate")&
            (pa_election["precinct"]=="Lower Moreland 4-1")&
            (pa_election["party"]=="DEM"), "candidate"] = "MARIA COLLETT"

In [96]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State Senate")&
            (pa_election["precinct"]=="Lower Gwynedd 1-2")&
            (pa_election['candidate']=="STEWART GREENLEAF JR"), "votes"] = 427

In [97]:
pa_election.loc[(pa_election["county"]=='Montgomery')&
            (pa_election["office"]=="State Senate")&
            (pa_election["precinct"]=="Lower Gwynedd 1-4")&
            (pa_election['candidate']=="MARIA COLLETT"), "votes"] = 584.0

In [98]:
pa_election[pa_election["county"]=="Franklin"]

,index,county,precinct,office,district,party,candidate,votes
202108,0,Franklin,Antrim Twp - First Dist,Registered Voters,NaN,NaN,NaN,1723
202109,1,Franklin,Antrim Twp - First Dist,Ballots Cast,NaN,NaN,NaN,1092
202110,2,Franklin,Antrim Twp - First Dist,Straight Party,NaN,DEM,DEMOCRATIC,96
202111,3,Franklin,Antrim Twp - First Dist,Straight Party,NaN,REP,REPUBLICAN,611
202112,4,Franklin,Antrim Twp - First Dist,Straight Party,NaN,GRN,GREEN,0
...,...,...,...,...,...,...,...,...
203850,1742,Franklin,West End Shippensburg,U.S. House,13.0,DEM,BRENT OTTAWAY,261
203851,1743,Franklin,West End Shippensburg,U.S. House,13.0,REP,JOHN JOYCE,221
203852,1744,Franklin,West End Shippensburg,U.S. House,13.0,NaN,WRITE-IN,5
203853,1745,Franklin,West End Shippensburg,State House,89.0,REP,ROB KAUFFMAN,296


In [99]:
pa_election[pa_election["precinct"]=="Upper Mahanoy Township (9th District)"]

,index,county,precinct,office,district,party,candidate,votes
103771,1352,Northumberland,Upper Mahanoy Township (9th District),Ballots Cast,NaN,NaN,NaN,7
103772,1353,Northumberland,Upper Mahanoy Township (9th District),Straight Party,NaN,DEM,NaN,1
103773,1354,Northumberland,Upper Mahanoy Township (9th District),Straight Party,NaN,REP,NaN,6
103774,1355,Northumberland,Upper Mahanoy Township (9th District),Straight Party,NaN,GRN,NaN,0
103775,1356,Northumberland,Upper Mahanoy Township (9th District),Straight Party,NaN,LIB,NaN,0
103776,1357,Northumberland,Upper Mahanoy Township (9th District),U.S. Senate,NaN,DEM,"BOB CASEY, JR",3
103777,1358,Northumberland,Upper Mahanoy Township (9th District),U.S. Senate,NaN,REP,LOU BARLETTA,11
103778,1359,Northumberland,Upper Mahanoy Township (9th District),U.S. Senate,NaN,GRN,NEAL GALE,0
103779,1360,Northumberland,Upper Mahanoy Township (9th District),U.S. Senate,NaN,LIB,"DALE R KERNS, JR",0
103780,1361,Northumberland,Upper Mahanoy Township (9th District),U.S. Senate,NaN,NaN,WRITE-IN,0


## Export Files

Delaware and Carbon are dealt with separately, because I remove duplicate rows in Delaware and drop 2 empty rows from the Carbon file

In [100]:
for county in list(pa_election["county"].unique()):
    if county not in ["Delaware","Carbon"]:
        # Filter down the modified file
        filtered = pa_election[pa_election["county"]==county].copy(deep = True)
        # Load in the original file
        test_load = pd.read_csv(path_in+"20181106__pa__general__"+county.lower()+"__precinct.csv")
        # Get the original column list
        og_column_list = list(test_load.columns)
        # Add in the original columns
        test_load.reset_index(inplace = True, drop = False)
        
        # We expect this assertion to fail in Berks and Montgomery b/c we added rows
        if county not in ["Montgomery", "Berks"]:
            print(county)
            print(filtered.shape[0])
            print(test_load.shape[0])
            assert(filtered.shape[0] - test_load.shape[0] == 0)

        # Test whether we need to rejoin data i.e if it had more columns than the ones we filtered down to
        if list(test_load.columns) == list(filtered.columns):
            filtered.drop(["index"], axis = 1, inplace = True)
            filtered = filtered[og_column_list]
            filtered.to_csv(path_out+"20181106__pa__general__"+county.lower()+"__precinct.csv", index = False)

        else:
            test_load.drop(["county","precinct","office","district","party","candidate","votes"], axis = 1, inplace = True)
            joined = pd.merge(filtered, test_load, how = "outer", indicator = True, on = "index")
            assert(joined[joined["_merge"]!="both"].shape[0]==0)
            joined.drop(["index","_merge"], axis = 1, inplace = True)
            joined = joined[og_column_list]
            joined.to_csv(path_out+"20181106__pa__general__"+county.lower()+"__precinct.csv", index = False)        

York
3856
3856
Philadelphia
30605
30605
Allegheny
23997
23997
Bedford
3119
3119
Butler
2123
2123
Lawrence
1722
1722
Cameron
220
220
Luzerne
3888
3888
Huntingdon
1508
1508
Clearfield
1470
1470
Lancaster
3622
3622
Armstrong
1404
1404
Crawford
1270
1270
Elk
450
450
Somerset
952
952
Lebanon
1467
1467
Warren
507
507
Dauphin
2955
2955
Pike
270
270
Forest
191
191
Jefferson
592
592
Lehigh
3338
3338
McKean
924
924
Cambria
3048
3048
Westmoreland
6771
6771
Lycoming
2150
2150
Northumberland
1537
1537
Clarion
800
800
Washington
3689
3689
Venango
1012
1012
Clinton
748
748
Wyoming
727
727
Indiana
929
929
Sullivan
183
183
Bucks
4273
4273
Erie
2461
2461
Adams
980
980
Fayette
1949
1949
Schuylkill
3914
3914
Chester
5441
5441
Wayne
560
560
Blair
3415
3415
Centre
2354
2354
Monroe
784
784
Susquehanna
4326
4326
Union
594
594
Tioga
902
902
Cumberland
2722
2722
Bradford
1525
1525
Potter
528
528
Greene
616
616
Fulton
325
325
Juniata
684
684
Northampton
4464
4464
Columbia
672
672
Mercer
2533
2533
Lackawanna
3965

In [101]:
for county in list(pa_election["county"].unique()):
    if county in ["Delaware"]:
        # Filter down the modified file
        filtered = pa_election[pa_election["county"]==county].copy(deep = True)

        # Load in the original file
        test_load = pd.read_csv(path_in+"20181106__pa__general__"+county.lower()+"__precinct.csv")
        og_column_list = list(test_load.columns)
        test_load.reset_index(inplace = True, drop = False)

        assert(filtered.shape[0] - test_load.shape[0] == 0)

        # Test whether we need to rejoin data
        if list(test_load.columns) == list(filtered.columns):
            print("Columns equal", county)
            filtered.drop(["index"], axis = 1, inplace = True)
            #print(filtered.head())
            #filtered.to_csv(path_out+"20181106__pa__general__"+county.lower()+"__precinct.csv", index = False)

        else:
            test_load.drop(["county","precinct","office","district","party","candidate","votes"], axis = 1, inplace = True)
            joined = pd.merge(filtered, test_load, how = "outer", indicator = True, on = "index")
            assert(joined[joined["_merge"]!="both"].shape[0]==0)
            joined.drop(["index","_merge"], axis = 1, inplace = True)
            joined = joined[og_column_list]
            joined.drop_duplicates(inplace = True)
            #print(joined.head())
            joined.to_csv(path_out+"20181106__pa__general__"+county.lower()+"__precinct.csv", index = False)  
            
for county in list(pa_election["county"].unique()):
    if county in ["Carbon"]:
        # Filter down the modified file
        filtered = pa_election[pa_election["county"]==county].copy(deep = True)

        # Load in the original file
        test_load = pd.read_csv(path_in+"20181106__pa__general__"+county.lower()+"__precinct.csv")
        og_column_list = list(test_load.columns)
        test_load.reset_index(inplace = True, drop = False)

        assert(filtered.shape[0] - test_load.shape[0] == 0)

        # Test whether we need to rejoin data
        if list(test_load.columns) == list(filtered.columns):
            print("Columns equal", county)
            filtered.drop(["index"], axis = 1, inplace = True)
            #print(filtered.head())
            #filtered.to_csv(path_out+"20181106__pa__general__"+county.lower()+"__precinct.csv", index = False)

        else:
            test_load.drop(["county","precinct","office","district","party","candidate","votes"], axis = 1, inplace = True)
            joined = pd.merge(filtered, test_load, how = "outer", indicator = True, on = "index")
            assert(joined[joined["_merge"]!="both"].shape[0]==0)
            joined.drop(["index","_merge"], axis = 1, inplace = True)
            joined = joined[og_column_list]
            joined.drop_duplicates(inplace = True)
            joined = joined[~joined["office"].isna()]
            #print(joined.head())
            joined.to_csv(path_out+"20181106__pa__general__"+county.lower()+"__precinct.csv", index = False) 